In [16]:
import sys
import os

# Determine the root directory (one level up from the notebook directory)
root_dir = os.path.abspath('..')

# Add the root directory to sys.path
sys.path.append(root_dir)


In [17]:
import pandas as pd

data = pd.read_csv('../../src/data/Hyperliquid Purty Palooza Winner List - Full Wallet Info - Sheet1.csv', usecols=['Wallet', 'HYPE_REWARD']).iloc[1:]
data['HYPE_REWARD'] = data['HYPE_REWARD'].astype(float)
data = data.dropna(subset=['Wallet'])

print(data.head())

                                       Wallet  HYPE_REWARD
1  0x0077bddbd1c19aedf79ccf40624ee8a18d331a4b       0.0795
2  0x023468c9c9d9b99444ddd614fa6581e6495aafef       0.0398
3  0x02ab9d831ba030997d1a84c62d150ce0aaf2a759       0.0066
4  0x046020bf59d0896371c0388baca5549e566b2f73       0.0284
5  0x058fd36a48e1c9980b34b41eac8a46c3eaf19a41       0.0253


In [18]:
data.tail()

,Wallet,HYPE_REWARD
177,0xf464c59f62fcba84ed508cf36bc5d2ae2a48877d,0.0507
178,0xf6f4fa2a5f79d00d31fcd016812c9fb42a99a5b2,0.0398
179,0xff6026607daea65a42cd441395d70c3f3418700a,0.1003
180,0xfddedd992fc3b3cec21a59f8dc2c1221c5a0ff67,0.0160
181,0xff1a1aa3f81986ce934f42be48488a6fddb38874,1.4241


In [19]:
# Filter data for specific wallet
row = data[data['Wallet'] == '0x21ad3f3d7b5770a62891bde06ca7db3d8b518dd5']
print(row)


                                        Wallet  HYPE_REWARD
23  0x21ad3f3d7b5770a62891bde06ca7db3d8b518dd5       8.1638


In [20]:
data['HYPE_REWARD'].sum()

85.99980000000001

In [21]:
import eth_account
from eth_account.signers.local import LocalAccount
from core.config import settings
from hyperliquid.exchange import Exchange
from hyperliquid.info import Info


def setup(base_url=None, skip_ws=False):
    account: LocalAccount = eth_account.Account.from_key(settings.HYPER_LIQUID_API_KEY)
    address = settings.HYPER_LIQUID_ADDRESS
    if address == "":
        address = account.address
    print("Running with account address:", address)
    if address != account.address:
        print("Running with agent address:", account.address)
    info = Info(base_url, skip_ws)
    user_state = info.user_state(address)
    spot_user_state = info.spot_user_state(address)
    margin_summary = user_state["marginSummary"]
    if (
        float(margin_summary["accountValue"]) == 0
        and len(spot_user_state["balances"]) == 0
    ):
        print("Not running the example because the provided account has no equity.")
        url = info.base_url.split(".", 1)[1]
        error_string = f"No accountValue:\nIf you think this is a mistake, make sure that {address} has a balance on {url}.\nIf address shown is your API wallet address, update the config to specify the address of your account, not the address of the API wallet."
        raise Exception(error_string)
    exchange = Exchange(account, base_url, account_address=address)
    return address, info, exchange

In [25]:
from hyperliquid.utils import constants

address, info, exchange = setup(constants.MAINNET_API_URL, skip_ws=True)
if exchange.account_address != exchange.wallet.address:
    raise Exception("Agents do not have permission to perform internal transfers")


Running with account address: 0x287cDA72b62189a9A2b83a7A369fdFAe8101BED7


In [27]:
from web3 import Web3

# Get initial balance
initial_balance = 86
remaining_balance = initial_balance

for index, row in data.iterrows():
    if pd.isnull(row["Wallet"]) or row["Wallet"] == "None" or row["Wallet"] == "":
        print(f"Skipping row {index} due to invalid wallet: {row['Wallet']}")
        continue
    
    hype_amount = float(row['HYPE_REWARD'])
    
    # Check if we have enough balance for this transfer
    if hype_amount > remaining_balance:
        print(f"❌ Insufficient balance for wallet {row['Wallet']}: Need {hype_amount} HYPE but only have {remaining_balance} HYPE")
        continue
        
    print(f"Processing transfer for wallet {row['Wallet']}: {hype_amount} HYPE")
    try:
        transfer_result = exchange.spot_transfer(
            hype_amount, Web3.to_checksum_address(row["Wallet"]), "HYPE:0x0d01dc56dcaaca66ad901c959b4011ec"
        )
        remaining_balance -= hype_amount
        print(f"✅ Transfer successful: {transfer_result}")
        print(f"Remaining balance: {remaining_balance} HYPE")
    except Exception as e:
        print(f"❌ Error transferring to {row['Wallet']}: {str(e)}")

Processing transfer for wallet 0x0077bddbd1c19aedf79ccf40624ee8a18d331a4b: 0.0795 HYPE


✅ Transfer successful: {'status': 'ok', 'response': {'type': 'default'}}
Remaining balance: 85.9205 HYPE
Processing transfer for wallet 0x023468c9c9d9b99444ddd614fa6581e6495aafef: 0.0398 HYPE
✅ Transfer successful: {'status': 'ok', 'response': {'type': 'default'}}
Remaining balance: 85.8807 HYPE
Processing transfer for wallet 0x02ab9d831ba030997d1a84c62d150ce0aaf2a759: 0.0066 HYPE
✅ Transfer successful: {'status': 'ok', 'response': {'type': 'default'}}
Remaining balance: 85.8741 HYPE
Processing transfer for wallet 0x046020bf59d0896371c0388baca5549e566b2f73: 0.0284 HYPE
✅ Transfer successful: {'status': 'ok', 'response': {'type': 'default'}}
Remaining balance: 85.8457 HYPE
Processing transfer for wallet 0x058fd36a48e1c9980b34b41eac8a46c3eaf19a41: 0.0253 HYPE
✅ Transfer successful: {'status': 'ok', 'response': {'type': 'default'}}
Remaining balance: 85.82039999999999 HYPE
Processing transfer for wallet 0x0833cc6673d4de9d81cfe467ba34803f43f13ecc: 1.4527 HYPE
✅ Transfer successful: {'statu